# Scraping des offres sur le site `marchespublics.sn`

Dans ce notebook, nous allons utiliser Selenium pour scraper les offres publiées sur le site [marchespublics.sn](http://www.marchespublics.sn/index.php?option=com_loffres&Itemid=104).

In [11]:
try:
    get_ipython().run_line_magic('pip', 'install selenium')
    get_ipython().run_line_magic('pip', 'install webdriver-manager')
    get_ipython().run_line_magic('pip', 'install pandas')
except NameError:
    pass

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime
import json

In [13]:
# Configurer Selenium avec ChromeDriver
options = Options()
options.add_argument('--headless')  # Exécuter en mode sans tête (sans interface graphique)
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

In [14]:
import os


In [15]:
# Configuration du driver avec le chemin local
chrome_driver_path = os.path.abspath("chromedriver-win64/chromedriver.exe")
print("Chemin absolu ChromeDriver:", chrome_driver_path)
assert os.path.exists(chrome_driver_path), f"ChromeDriver introuvable à {chrome_driver_path}"
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=options)

Chemin absolu ChromeDriver: c:\Users\SURFACE STUDIO\OneDrive\Bureau\Stage Neurotech\IA CRM\projet-neurotech\nouveau depart\chromedriver-win64\chromedriver.exe


In [16]:
# URL des offres "Prestation de services"
url = "http://www.marchespublics.sn/index.php?option=com_loffres&task=view&idcat=002&Itemid=104&gestion=2025&statut=1"
driver.get(url)


In [17]:
# definir l'année courante
current_year = datetime.now().year
# Trouver le tableau contenant les offres
rows = driver.find_elements(By.CSS_SELECTOR, "table.content_table tbody tr")[1:]  # pour sauter l'entête


In [18]:
data = []

for row in rows[1:]:
    try:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) < 6:
            continue

        # ✅ Tenter de récupérer le lien AVANT de continuer
        try:
            detail_td = cols[5]
            div = detail_td.find_element(By.TAG_NAME, "div")
            a_tags = div.find_elements(By.TAG_NAME, "a")
            if not a_tags:
                continue  # Pas de lien => ce n'est pas une ligne d'offre
            detail_link = a_tags[0].get_attribute("href")
        except:
            continue  # Évite les erreurs sur les lignes non valides

        # ✅ Extraire les autres infos de la ligne
        ref = cols[0].text.strip() or "NA"
        objet = cols[1].text.strip() or "NA"
        autorite = cols[2].text.strip() or "NA"
        publie_le = cols[3].text.strip() or "NA"
        date_limite = cols[4].text.strip() or "NA"

        # ✅ Ouvrir le lien dans un nouvel onglet
        driver.execute_script("window.open(arguments[0]);", detail_link)
        driver.switch_to.window(driver.window_handles[1])

        try:
            table_elem = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table.no-style"))
            )
            td_elem = table_elem.find_element(By.TAG_NAME, "td")
            detail_text = td_elem.text.strip() or "NA"
        except Exception as e:
            print(f"⚠️ Détail non trouvé pour {ref} : {e}")
            detail_text = "NA"

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

        data.append([ref, objet, autorite, publie_le, date_limite, detail_text])

    except Exception as e:
        print(f"❌ Erreur sur l’offre {ref if 'ref' in locals() else 'inconnue'} : {e}")
        continue

driver.quit()


In [19]:
print("Nombre d'offres récupérées :", len(data))

Nombre d'offres récupérées : 6


In [9]:
# le dataframe
df = pd.DataFrame(data, columns=["Référence", "Objet", "Autorité", "Publié le", "Date limite", "Détail"])
df

,Référence,Objet,Autorité,Publié le,Date limite,Détail
0,S_LMDG_160,Révisions des 12 000 heures des moteurs cummin...,Port Autonome de Dakar (PAD),17/06/2025,16/07/2025,REPUBLIQUE DU SENEGAL N°…………PAD/DG/SG/LMDG\n ...
1,S_TDS_011,Frais d'impression et de communication Marché ...,Société de Télédiffusion du Sénégal (TDS.sa),18/06/2025,25/06/2025,NA
2,S_DGPPE_030,Service de couverture de la police d'assurance...,"Ministère de l’Economie, du Plan et de la Coop...",18/06/2025,02/07/2025,SereR SENTER\nSeneggisc\nMARAHEN RECYCLAGE\ndé...
3,S_DAF_038,Sélection d'un prestataire pour l'organisation...,Pharmacie Nationale d'Approisionnement(PNA),19/06/2025,04/07/2025,Section 0 : Avis de DRPCO – Cas sans pré quali...
4,S_Dd_075,Marché de clientèle pour la maintenance des di...,Port Autonome de Dakar (PAD),03/06/2025,09/07/2025,Avis d’Appel d’Offres – Cas sans pré qualifica...
5,S_ BOCS_015,Sélection d’une compagnie d’assurance pour la ...,Bureau Opérationnel de Coordination et de Suiv...,28/05/2025,26/06/2025,Avis d'Appel d'offres (AA0)\nBureau Opérationn...


In [10]:
offres_liste = df.to_dict(orient='records')
offres_liste

[{'Référence': 'S_LMDG_160',
  'Objet': 'Révisions des 12 000 heures des moteurs cummins de la chaloupe BEER',
  'Autorité': 'Port Autonome de Dakar (PAD)',
  'Publié le': '17/06/2025',
  'Date limite': '16/07/2025',
  'Détail': "REPUBLIQUE DU SENEGAL  N°…………PAD/DG/SG/LMDG\n -------------------  \nMINISTERE DES PECHES ET Dakar, le..........………........\nDES INFRASTRUCTURES MARITIMES\nET PORTUAIRES     \n -------------------\nPORT AUTONOME DE DAKAR  \n 21, Boulevard de la Libération\nB.P. 3195 DAKAR – SENEGAL\n& : (221) 849.45.45\nFAX: : (221) 823.36.06\nTELEX: 210404 PADKR\nNINEA : 00 199 86-2-G-3\nAVIS DE LA DRPCO N° S_LMDG_160\n==================\nRévisions des 12000 heures des moteurs des chaloupes « Coumba Castel et Beer »\n==================\n1. Le présent avis de la DRPCO fait suite à l’Avis Général de Passation des Marchés paru dans le journal « LE SOLEIL » n°16380 du 09 janvier 2025, publié dans le Portail des Marchés publics du Sénégal et sur le site du Port au www.portdakar.sn

In [23]:
offres_liste[0]

{'Référence': 'S_Dd_075',
 'Objet': 'Marché de clientèle pour la maintenance des dispositifs de sécurité physiques avec des pièces de rechange (Marché de clientèle)',
 'Autorité': 'Port Autonome de Dakar (PAD)',
 'Publié le': '03/06/2025',
 'Date limite': '09/07/2025',
 'Détail': 'Avis d’Appel d’Offres – Cas sans pré qualification\nAvis d’Appel d’Offres Ouvert N° : S_Dd_075\n==================\nPORT AUTONOME DE DAKAR\nMarché de clientèle pour la maintenance des dispositifs\nde Sécurité Physique avec des pièces de rechange\n==================\n1. Le présent avis d’appel d’offres fait suite à l’Avis Général de Passation des Marchés paru dans le journal « LE SOLEIL » du 09 janvier 2025 et publié dans le Portail des Marchés publics du Sénégal ainsi que sur le site du Port au www.portdakar.sn.\n1. Le Port Autonome de Dakar a obtenu dans le cadre de son budget de fonctionnement 2025 des fonds nécessaires afin de financer le marché de clientèle pour la maintenance des dispositifs de Sécurité 

In [24]:
# Convertir le dictionnaire en format texte
offres_texte = json.dumps(offres_liste, indent=4, ensure_ascii=False)
print(offres_texte)

[
    {
        "Référence": "S_Dd_075",
        "Objet": "Marché de clientèle pour la maintenance des dispositifs de sécurité physiques avec des pièces de rechange (Marché de clientèle)",
        "Autorité": "Port Autonome de Dakar (PAD)",
        "Publié le": "03/06/2025",
        "Date limite": "09/07/2025",
        "Détail": "Avis d’Appel d’Offres – Cas sans pré qualification\nAvis d’Appel d’Offres Ouvert N° : S_Dd_075\n==================\nPORT AUTONOME DE DAKAR\nMarché de clientèle pour la maintenance des dispositifs\nde Sécurité Physique avec des pièces de rechange\n==================\n1. Le présent avis d’appel d’offres fait suite à l’Avis Général de Passation des Marchés paru dans le journal « LE SOLEIL » du 09 janvier 2025 et publié dans le Portail des Marchés publics du Sénégal ainsi que sur le site du Port au www.portdakar.sn.\n1. Le Port Autonome de Dakar a obtenu dans le cadre de son budget de fonctionnement 2025 des fonds nécessaires afin de financer le marché de clientèl